In [1]:
import pandas as pd
import pickle
import datetime
import numpy as np
from math import radians, cos, sin, asin, sqrt
from sklearn.cluster import KMeans
import telebot

In [2]:
df = pd.read_csv('test.csv')
df

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N
...,...,...,...,...,...,...,...,...,...
625129,id3008929,1,2016-01-01 00:02:52,1,-74.003464,40.725105,-74.001251,40.733643,N
625130,id3700764,1,2016-01-01 00:01:52,1,-74.006363,40.743782,-73.953407,40.782467,N
625131,id2568735,1,2016-01-01 00:01:24,2,-73.972267,40.759865,-73.876602,40.748665,N
625132,id1384355,1,2016-01-01 00:00:28,1,-73.976501,40.733562,-73.854263,40.891788,N


In [22]:
with open("model2.pkl", "rb") as f:
    model = pickle.load(f)

In [4]:


df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['periods'] = (((df['pickup_datetime'].dt.hour*60 + df['pickup_datetime'].dt.minute))/10).round()

def period_of_day(x):
    if x.hour >= 4 and x.hour <= 10:
        return 0
    elif x.hour >= 10 and x.hour <= 16:
        return 1
    elif x.hour >= 16 and x.hour <= 22:
        return 3
    elif x.hour > 22 and x.hour < 4:
        return 'night'
    
df['pickup_period']=df['pickup_datetime'].apply(lambda x :period_of_day(datetime.datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S").time()) )
df['pickup_period'].isna()

0         True
1         True
2         True
3         True
4         True
          ... 
625129    True
625130    True
625131    True
625132    True
625133    True
Name: pickup_period, Length: 625134, dtype: bool

In [5]:
df['pickup_period'].fillna(4, inplace = True)

In [6]:
def distance(df):
    lat1, lon1, lat2, lon2 = df.pickup_latitude, df.pickup_longitude, df.dropoff_latitude, df.dropoff_longitude 
    R = 3959.87433 #hрадиус земли в милях
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
    return R * c

df['distance_line_km'] = df.apply(lambda x: distance(x), axis = 1)

In [7]:
df

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,periods,pickup_period,distance_line_km
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,144.0,4.0,1.707032
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,144.0,4.0,1.714996
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,144.0,4.0,0.811837
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N,144.0,4.0,3.274984
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N,144.0,4.0,0.597208
...,...,...,...,...,...,...,...,...,...,...,...,...
625129,id3008929,1,2016-01-01 00:02:52,1,-74.003464,40.725105,-74.001251,40.733643,N,0.0,4.0,0.601308
625130,id3700764,1,2016-01-01 00:01:52,1,-74.006363,40.743782,-73.953407,40.782467,N,0.0,4.0,3.851317
625131,id2568735,1,2016-01-01 00:01:24,2,-73.972267,40.759865,-73.876602,40.748665,N,0.0,4.0,5.067918
625132,id1384355,1,2016-01-01 00:00:28,1,-73.976501,40.733562,-73.854263,40.891788,N,0.0,4.0,12.667588


In [19]:
X2 = np.array(df[['dropoff_longitude', 'dropoff_latitude']], dtype='float64')
k = 10
model1 = KMeans(n_clusters=k, random_state=17).fit(X2)
pred2 = model1.predict(X2)
df['KMean2'] = pred2
df

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,periods,pickup_period,distance_line_km,KMean2,KMean
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,144.0,4.0,1.707032,8,0
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,144.0,4.0,1.714996,6,6
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,144.0,4.0,0.811837,0,0
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N,144.0,4.0,3.274984,0,3
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N,144.0,4.0,0.597208,8,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625129,id3008929,1,2016-01-01 00:02:52,1,-74.003464,40.725105,-74.001251,40.733643,N,0.0,4.0,0.601308,0,6
625130,id3700764,1,2016-01-01 00:01:52,1,-74.006363,40.743782,-73.953407,40.782467,N,0.0,4.0,3.851317,2,0
625131,id2568735,1,2016-01-01 00:01:24,2,-73.972267,40.759865,-73.876602,40.748665,N,0.0,4.0,5.067918,7,9
625132,id1384355,1,2016-01-01 00:00:28,1,-73.976501,40.733562,-73.854263,40.891788,N,0.0,4.0,12.667588,9,0


In [20]:
X = np.array(df[['pickup_longitude', 'pickup_latitude']], dtype='float64')
k = 10
model2 = KMeans(n_clusters=k, random_state=17).fit(X)
pred = model2.predict(X)
df['KMean'] = pred
df

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,periods,pickup_period,distance_line_km,KMean2,KMean
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,144.0,4.0,1.707032,8,0
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,144.0,4.0,1.714996,6,6
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,144.0,4.0,0.811837,0,0
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N,144.0,4.0,3.274984,0,3
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N,144.0,4.0,0.597208,8,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625129,id3008929,1,2016-01-01 00:02:52,1,-74.003464,40.725105,-74.001251,40.733643,N,0.0,4.0,0.601308,0,6
625130,id3700764,1,2016-01-01 00:01:52,1,-74.006363,40.743782,-73.953407,40.782467,N,0.0,4.0,3.851317,2,0
625131,id2568735,1,2016-01-01 00:01:24,2,-73.972267,40.759865,-73.876602,40.748665,N,0.0,4.0,5.067918,7,9
625132,id1384355,1,2016-01-01 00:00:28,1,-73.976501,40.733562,-73.854263,40.891788,N,0.0,4.0,12.667588,9,0


In [30]:
df['predicted'] = np.exp(model.predict(df.drop(['id', 'vendor_id', 'pickup_datetime', 'store_and_fwd_flag'], axis = 1)))

ValueError: could not convert string to float: 'N'

In [26]:
prog['id'] = pd.DataFrame(df['id'])
prog['predicted'] = df['predicted']
prog

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

,Unnamed: 0,id,predicted
0,0,id3004672,1465.929123
1,1,id3505355,1517.604278
2,2,id1217141,1419.794716
3,3,id2150126,1478.474187
4,4,id1598245,1522.344796
...,...,...,...
625129,625129,id3008929,673.589934
625130,625130,id3700764,681.534292
625131,625131,id2568735,677.564734
625132,625132,id1384355,781.827313
